# Capstone Project. **Real Estate Valuation Preprocessing**  
#### **Angela Brito**
#### Bachelor in Data and Business Analytics


Libraries

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import scipy.stats as stats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

file_path = '/content/drive/MyDrive/idealista_madrid.csv'
data = pd.read_csv(file_path)

In [4]:

data.head()

,url,listingUrl,title,id,price,baths,rooms,sqft,description,address,typology,advertiserProfessionalName,advertiserName
0,https://www.idealista.com/inmueble/104027174/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Villanueva,104027174,1920000,3,3,183,Residencia única con acabados de la más alta c...,"Recoletos, Madrid",Pisos,Promora Madrid,Promora Madrid
1,https://www.idealista.com/inmueble/102321942/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Núñez de Balboa,102321942,1995000,3,3,170,"Preciosa reforma a estrenar, con terrazas y ga...","Castellana, Madrid",Pisos,Madrid MMC,Engel & Völkers Madrid
2,https://www.idealista.com/inmueble/103334142/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Conde Orgaz-Piovera,103334142,1300000,3,4,270,DE SALAS CONSULTORES INMOBILIARIOS MAV02679 le...,"Hortaleza, Madrid",Pisos,De Salas Consultores Inmobiliarios,De Salas Consultores Inmobiliarios
3,https://www.idealista.com/inmueble/104161987/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Nueva España,104161987,1650000,3,3,248,Gilmar Real Estate vende espectacular piso en ...,"Chamartín, Madrid",Pisos,departamento comercial,Gilmar Viso - Chamartín
4,https://www.idealista.com/inmueble/103989666/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Claudio Coello,103989666,1590750,3,3,116,Descubre esta impresionante vivienda exterior ...,"Recoletos, Madrid",Pisos,Walter Haus Salamanca,Walter Haus Madrid


In [5]:
for i in data.columns:
  print(i)

url
listingUrl
title
id
price
baths
rooms
sqft
description
address
typology
advertiserProfessionalName
advertiserName


# **Preprocessing Steps**

## **Step 1. Handle Missing Values**

In [6]:

print(data.isnull().sum())

url                           0
listingUrl                    0
title                         0
id                            0
price                         0
baths                         0
rooms                         0
sqft                          0
description                   1
address                       0
typology                      0
advertiserProfessionalName    0
advertiserName                0
dtype: int64


In [7]:
missing_data = data.isnull().sum()
missing_data = missing_data[missing_data > 0]
print(missing_data)


description    1
dtype: int64


In [8]:
import pandas as pd

data = data.dropna()

*Therefore, description is the only variable with missing data in my dataset, with 1 missing value.*  

***Since it is an important variable, we will directly remove it***

---



## **Step 2. Handle Duplicate Values**

In [9]:
import pandas as pd

if 'id' in data.columns:
    duplicate_rows = data[data.duplicated('id', keep=False)]

    duplicate_rows = duplicate_rows.sort_values('id')

    print("Duplicate entries based on ID:\n", duplicate_rows)
else:
    print("The 'ID' column does not exist in your DataFrame.")


Duplicate entries based on ID:
 Empty DataFrame
Columns: [url, listingUrl, title, id, price, baths, rooms, sqft, description, address, typology, advertiserProfessionalName, advertiserName]
Index: []


Therefore, there are no duplicate values in my dataset

##**Step 3. Feature Engineering**

**Feature Engineering**: *process of extracting new variables from existing data*

I have chosen to delve deeper into the "Description" variable to see if i can add extra layers that can help with the valuation of certain properties. I will be looking for features including **garage, terrace, pool, garden, etc.**

This can help by:

*   **Improving Model Accuracy:** by adding new and additional information to the models, helping them better and more accurate predictions ist item
*   **Enhance Data Understanding:** increasing the information available within the dataset allows for deeper insights
*   **Increase Predictive Power:** good use of feature engineering can help increase the complexity of the models, hence increasing their value


In [10]:
import pandas as pd
import re

In [11]:
data['description'] = data['description'].str.lower().str.replace('[^\w\s]', '')
features = {
    'Terrace': 'terraza|balcón|porche',
    'Garage': 'garaje|aparcamiento',
    'Pool': 'piscina',
    'Garden': 'jardín',
    'Exterior': 'exterior',
    'RecentlyRenovated': 'reformo|renovo|renovada|reformada recientemente',
    'HighFloor': 'planta alta|última planta',
    'NaturalLight': 'luminoso|mucha luz',
    'Views': 'vistas',
    'ClimateControl': 'calefacción|aire acondicionado',
    'EnergyEfficient': 'eficiencia energética',
    'Security': 'vigilancia|alarma|seguridad',
    'Accessible': 'acceso para discapacitados|ascensor',
    'SouthFacing': 'sur',
    'EastFacing': 'este',
    'NorthFacing': 'norte',
    'WestFacing': 'oeste',
    'NearSchool': 'colegio|escuela|instituto',
    'NearAmenities': 'centro comercial|supermercado|mercado',
    'NearMetro':'metro|estación de metro',
    'NearLandmark':'palacio real|museo del prado|prado|el retiro|parque del retiro|plaza mayor|puerta del sol|santiago bernabéu|estadio santiago bernabéu|templo de debod|mercado de san miguel|museo reina sofia|gran via'
}

for feature_name, pattern in features.items():
    data[feature_name] = data['description'].apply(lambda x: bool(re.search(pattern, str(x), re.IGNORECASE)))

data.head()


<ipython-input-11-13d5396d6d8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['description'] = data['description'].str.lower().str.replace('[^\w\s]', '')


,url,listingUrl,title,id,price,baths,rooms,sqft,description,address,...,Security,Accessible,SouthFacing,EastFacing,NorthFacing,WestFacing,NearSchool,NearAmenities,NearMetro,NearLandmark
0,https://www.idealista.com/inmueble/104027174/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Villanueva,104027174,1920000,3,3,183,residencia única con acabados de la más alta c...,"Recoletos, Madrid",...,False,False,True,True,False,False,False,False,False,True
1,https://www.idealista.com/inmueble/102321942/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Núñez de Balboa,102321942,1995000,3,3,170,"preciosa reforma a estrenar, con terrazas y ga...","Castellana, Madrid",...,False,False,False,True,False,False,False,False,True,False
2,https://www.idealista.com/inmueble/103334142/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Conde Orgaz-Piovera,103334142,1300000,3,4,270,de salas consultores inmobiliarios mav02679 le...,"Hortaleza, Madrid",...,False,False,True,True,False,False,False,False,False,False
3,https://www.idealista.com/inmueble/104161987/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Nueva España,104161987,1650000,3,3,248,gilmar real estate vende espectacular piso en ...,"Chamartín, Madrid",...,False,False,False,True,False,False,False,False,True,False
4,https://www.idealista.com/inmueble/103989666/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Claudio Coello,103989666,1590750,3,3,116,descubre esta impresionante vivienda exterior ...,"Recoletos, Madrid",...,False,False,False,True,False,False,False,False,False,False


Group variable **address** the 21 main districts of Madrid based on Ayuntamiento de Madrid, Mas Madrid y otros: https://www.madrid.es/portales/munimadrid/es/Inicio/Movilidad-y-transportes/Circulacion-y-aparcamiento/Distritos-y-barrios/?vgnextfmt=default&vgnextoid=5a86befb3fd31810VgnVCM2000001f4a900aRCRD&vgnextchannel=b3128fb9458fe410VgnVCM1000000b205a0aRCRD

https://madridfilmoffice.com/21-distritos/
https://masmadrid.org/distritos-madrid/mas-madrid-moratalaz/#:~:text=Consta%20de%20seis%20barrios%3A%20Pavones,les%20confiere%20una%20identidad%20propia.



In [12]:
neighborhoods_to_districts = {
    'Arganzuela': ['Arganzuela', 'Imperial', 'Acacias', 'Chopera', 'Legazpi', 'Delicias', 'Palos de Moguer', 'Atocha'],
    'Barajas': ['Barajas', 'Alameda de Osuna', 'Aeropuerto', 'Casco Historico de Barajas', 'Timon', 'Corralejos'],
    'Carabanchel': ['Carabanchel', 'San Isidro', 'Comillas', 'Opañel', 'Vista Alegre', 'Puerta Bonita', 'Buena Vista', 'Pau de Carabanchel', 'Abrantes'],
    'Centro': ['Centro', 'Palacio', 'Embajadores', 'Cortes', 'Cortes - Jeronimos', 'Justicia', 'Universidad', 'Sol', 'Sol Palacio', 'Sol Cortes'],
    'Chamartin': ['Chamartin', 'El Viso', 'Prosperidad', 'Ciudad Jardin', 'Hispanoamerica', 'Nueva España', 'Castilla'],
    'Chamberi': ['Chamberi', 'Gaztambide', 'Arapiles', 'Trafalgar', 'Almagro', 'Rios Rosas', 'Vallehermoso'],
    'Ciudad Lineal': ['Ciudad Lineal', 'Costillares', 'Ventas', 'Pueblo Nuevo', 'Quintana', 'Concepcion', 'San Pascual', 'San Juan Bautista', 'Colina', 'Atalaya'],
    'Fuencarral-El Pardo': ['Fuencarral', 'Mirasierra', 'Fuentelarreina', 'El Pardo', 'Fuencarral-El Pardo', 'Pilar', 'La Paz', 'Peñagrande'],
    'Hortaleza': ['Hortaleza', 'Manoteras', 'Sanchinarro', 'Valdebebas', 'Palomas', 'La Piovera', 'Canillas', 'Pinar del Rey', 'Apostol Santiago', 'Valdefuentes', 'Valdebebas-Valdefuentes', 'Conde Orgaz-Piovera'],
    'Latina': ['Latina', 'Los Carmenes', 'Puerta del Angel', 'Campamento'],
    'Moncloa-Aravaca': ['Aravaca', 'El Plantio', 'Moncloa', 'Moncloa-Aravaca', 'Casa de Campo', 'Argüelles', 'Ciudad Universitaria', 'Valdezarza', 'Valdemarin'],
    'Moratalaz': ['Moratalaz', 'Pavones', 'Horcajo', 'Marroquina', 'Media Legua', 'Fontarron', 'Vinateros'],
    'Puente de Vallecas': ['Vallecas', 'Puente de Vallecas', 'Portazgo', 'San Diego', 'Numancia', 'Entrevias-El Pozo', 'Madrid Sur', 'Palomeras Sureste'],
    'Retiro': ['Retiro', 'Pacífico', 'Adelfas', 'Estrella', 'Ibiza', 'Jerónimos', 'Niño Jesús'],
    'Salamanca': ['Salamanca', 'Lista', 'Castellana', 'Recoletos', 'Goya', 'Fuente del Berro', 'Guindalera'],
    'San Blas-Canillejas': ['San Blas', 'Canillejas', 'San Blas-Canillejas', 'Salvador', 'Simancas', 'Hellin', 'Amposta', 'Arcos', 'Rosas', 'Rejas', 'El Salvador'],
    'Tetuan': ['Tetuan', 'Bellas Vistas', 'Cuatro Caminos', 'Castillejos', 'Almenara', 'Valdeacederas', 'Berruguete'],
    'Usera': ['Usera', 'Pradolongo', 'Orcasur', 'San Fermin', 'Almendrales', 'Moscardo', 'Zofio', 'Orcasitas'],
    'Vicalvaro': ['Vicalvaro', 'Valdebernardo', 'Valderrivas', 'El Cañaveral', 'Casco historico de Vicalvaro'],
    'Villa de Vallecas': ['Villa de Vallecas', 'Vallecas', 'Santa Eugenia', 'Ensanche de Vallecas', 'Casco historico de Vallecas'],
    'Villaverde': ['Villaverde', 'Villaverde Alto', 'Villaverde Bajo', 'Butarque', 'Ciudad de los Angeles', 'Los Angeles', 'San Cristobal']
}

district_mapping = {neighborhood: district for district, neighborhoods in neighborhoods_to_districts.items() for neighborhood in neighborhoods}


In [13]:
import unicodedata

def remove_accents(input_str):
    """
    Removes accents from a given string.
    """
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])


In [14]:
def map_address_to_district(address):
    address_norm = remove_accents(address).lower()
    for neighborhood, district in district_mapping.items():
        neighborhood_norm = remove_accents(neighborhood).lower()
        if neighborhood_norm in address_norm:
            return district
    return "Other"

In [15]:

data['District'] = data['address'].apply(map_address_to_district)

In [16]:

data['District'].value_counts()

District
Salamanca              162
Centro                 139
Chamberi                98
Chamartin               94
Hortaleza               86
Moncloa-Aravaca         80
Fuencarral-El Pardo     47
Retiro                  44
Tetuan                  43
Ciudad Lineal           43
San Blas-Canillejas     22
Arganzuela              18
Carabanchel             10
Latina                   6
Usera                    6
Barajas                  6
Villaverde               3
Villa de Vallecas        3
Puente de Vallecas       2
Vicalvaro                1
Moratalaz                1
Name: count, dtype: int64

In [17]:
def debug_map_address_to_district(address):
    address_norm = remove_accents(address).lower()
    for neighborhood, district in district_mapping.items():
        neighborhood_norm = remove_accents(neighborhood).lower()
        if neighborhood_norm in address_norm:
            return district
    print(address)
    return "Other"

data['debug_district'] = data['address'].apply(debug_map_address_to_district)


No values that have no district !!! YAY

In [18]:
data['price_per_sqft'] = data['price'] / data['sqft']

In [19]:
for i in data.columns:
  print(i)

url
listingUrl
title
id
price
baths
rooms
sqft
description
address
typology
advertiserProfessionalName
advertiserName
Terrace
Garage
Pool
Garden
Exterior
RecentlyRenovated
HighFloor
NaturalLight
Views
ClimateControl
EnergyEfficient
Security
Accessible
SouthFacing
EastFacing
NorthFacing
WestFacing
NearSchool
NearAmenities
NearMetro
NearLandmark
District
debug_district
price_per_sqft


**Now let's count to see how many data points include the features to see whether it is worth while.**

In [20]:
feature_counts = {}

for feature in features.keys():
    feature_counts[feature] = data[feature].sum()


for feature, count in feature_counts.items():
    print(f"{feature}: {count}")


Terrace: 474
Garage: 437
Pool: 273
Garden: 195
Exterior: 427
RecentlyRenovated: 14
HighFloor: 54
NaturalLight: 295
Views: 270
ClimateControl: 439
EnergyEfficient: 14
Security: 224
Accessible: 285
SouthFacing: 177
EastFacing: 608
NorthFacing: 91
WestFacing: 134
NearSchool: 329
NearAmenities: 233
NearMetro: 570
NearLandmark: 242


In [21]:
data.price_per_sqft.describe()

count      914.000000
mean      6403.208191
std       2800.958358
min       1198.083067
25%       4396.135266
50%       5763.646461
75%       7944.729547
max      21299.638989
Name: price_per_sqft, dtype: float64

## **Step 3. Detecting and Handling Outliers**

#**Analysis 1**
###**Outliers based on location**



Group Data by Location

In [22]:
grouped = data.groupby('District')

In [23]:
import pandas as pd

def identify_outliers(group):
    Q1 = group['price_per_sqft'].quantile(0.25)
    Q3 = group['price_per_sqft'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_mask = (group['price_per_sqft'] < lower_bound) | (group['price_per_sqft'] > upper_bound)
    return outliers_mask


In [24]:
data['is_outlier'] = data.groupby('District').apply(identify_outliers).reset_index(level=0, drop=True)
outliers_by_district = data[data['is_outlier']].groupby('District').size()

print("Outliers by District:")
outliers_by_district

Outliers by District:


District
Centro                 9
Chamartin              1
Fuencarral-El Pardo    1
Moncloa-Aravaca        2
Retiro                 3
Salamanca              3
San Blas-Canillejas    2
Tetuan                 1
dtype: int64

## **Step 4. Data Enrichment**

Add the following sociodemographic variables:

*   Population density
*   Median household income
*   Crime rates
*   Access to public transportation and amenities
*   Education levels
*   Employment rates

From the Ayuntamiento de Madrid: https://www.madrid.es/portales/munimadrid/es/Inicio/El-Ayuntamiento/Estadistica/Distritos-en-cifras/Distritos-en-cifras-Informacion-de-Distritos-/?vgnextfmt=default&vgnextoid=74b33ece5284c310VgnVCM1000000b205a0aRCRD&vgnextchannel=27002d05cb71b310VgnVCM1000000b205a0aRCRD


In [25]:
import pandas as pd

excel_path = '/content/drive/MyDrive/TFG/Dataset/Sociodemographic_Final.xlsx'

socio_data = pd.read_excel(excel_path)

In [26]:
socio_data.head()


,,Ciudad,Centro,Arganzuela,Retiro,Salamanca,Chamartin,Tetuan,Chamberi,Fuencarral-El Pardo,...,Usera,Puente de Vallecas,Moratalaz,Ciudad Lineal,Hortaleza,Villaverde,Villa de Vallecas,Vicalvaro,San Blas-Canillejas,Barajas
0,Superficie (Ha.),60445.520000,522.820000,646.220000,546.620000,539.240000,917.550000,537.470000,467.920000,23783.840000,...,777.770000,1496.860000,610.320000,1142.570000,2762.610000,2018.760000,5146.720000,3526.670000,2229.240000,4171.650000
1,Densidad (hab./Ha.),55.255228,267.179909,237.231902,215.722074,270.198798,157.807204,297.694755,295.358181,10.445874,...,183.532407,161.406544,152.074322,192.850329,71.812887,78.780043,22.830269,23.762927,72.320163,11.661093
2,Edad media de la población,44.370000,44.020000,45.400000,47.580000,46.230000,45.750000,44.080000,46.290000,43.750000,...,42.610000,43.300000,48.260000,46.010000,42.800000,42.110000,39.960000,40.920000,43.800000,43.340000
3,Personas con nacionalidad española (01/01/2022),0.829180,0.733755,0.889494,0.909878,0.842645,0.894507,0.784828,0.866625,0.902283,...,0.750375,0.778057,0.889047,0.819356,0.871783,0.758441,0.837797,0.820486,0.844007,0.895058
4,Personas con nacionalidad extranjera (01/01/2022),0.141000,0.266202,0.106155,0.088027,0.155674,0.102558,0.199110,0.126733,0.089014,...,0.236035,0.197249,0.106385,0.151408,0.111200,0.208799,0.139361,0.126104,0.140883,0.099926


In [27]:
socio_df = socio_data.transpose()

socio_df.columns = socio_df.iloc[0]
socio_df = socio_df[1:]

socio_df.reset_index(inplace=True)
socio_df.rename(columns={'index': 'District'}, inplace=True)


In [28]:
socio_df.head()

,District,Superficie (Ha.),Densidad (hab./Ha.),Edad media de la población,Personas con nacionalidad española (01/01/2022),Personas con nacionalidad extranjera (01/01/2022),Nº total de hogares,Tamaño medio del hogar,Tasa de crecimiento demográfico 2021 (%),Renta neta media anual de los hogares Urban Audit (2020),Paro registrado (nº de personas registradas en SEPE en febrero 2023),Año medio de construcción de inmuebles de uso residencial,No sabe leer ni escribir o sin estudios,Primaria incompleta,Superficie media construida (m2) inmuebles de uso residencial,Número de inmuebles de uso residencial,Valor catastral medio de los bienes inmuebles: personas físicas en miles de €,En centros PRIVADOS CONCERTADOS,En centros PRIVADOS SIN CONCIERTO,En centros PÚBLICOS
0,Ciudad,60445.52,55.255228,44.37,0.82918,0.141,1.0,2.501912,-0.78,43003.0,148294.0,1973.206107,0.040039,0.101188,115.48855,1502190.0,90.421527,0.421171,0.181379,0.39745
1,Centro,522.82,267.179909,44.02,0.733755,0.266202,0.053231,1.99754,-1.11,36984.0,0.04456,1926.0,0.023226,0.063849,0.927942,0.05419,1.257932,0.360149,0.101329,0.538522
2,Arganzuela,646.22,237.231902,45.4,0.889494,0.106155,0.050032,2.322374,-1.05,45310.0,0.038592,1976.285714,0.020411,0.068992,0.833725,0.050177,0.926003,0.291119,0.194917,0.513964
3,Retiro,546.62,215.722074,47.58,0.909878,0.088027,0.037356,2.397897,-0.75,56302.0,0.023069,1964.666667,0.015502,0.051691,1.118437,0.03665,1.452984,0.551045,0.124804,0.324151
4,Salamanca,539.24,270.198798,46.23,0.842645,0.155674,0.048139,2.300152,-0.38,60401.0,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171


In [29]:
print(socio_df.columns)

Index(['District', 'Superficie (Ha.) ', 'Densidad (hab./Ha.) ',
       'Edad media de la población',
       'Personas con nacionalidad española (01/01/2022)',
       'Personas con nacionalidad extranjera (01/01/2022)',
       'Nº total de hogares', 'Tamaño medio del hogar',
       'Tasa de crecimiento demográfico 2021 (%)',
       'Renta neta media anual de los hogares Urban Audit (2020)',
       'Paro registrado (nº de personas registradas en SEPE en febrero 2023)',
       'Año medio de construcción de inmuebles de uso residencial ',
       'No sabe leer ni escribir o sin estudios', 'Primaria incompleta',
       'Superficie media construida (m2) inmuebles de uso residencial  ',
       'Número de inmuebles de uso residencial  ',
       'Valor catastral medio de los bienes inmuebles: personas físicas en miles de €',
       'En centros PRIVADOS CONCERTADOS', 'En centros PRIVADOS SIN CONCIERTO',
       'En centros PÚBLICOS'],
      dtype='object', name=' ')


In [30]:
new_column_names = {
    'Superficie (Ha.) ': 'Surface',
    'Densidad (hab./Ha.) ': 'Density',
    'Edad media de la población': 'Population Median Age',
    'Personas con nacionalidad española (01/01/2022)': 'People with Spanish Nationality',
    'Personas con nacionalidad extranjera (01/01/2022)': 'People with Foreign Nationality',
    'Nº total de hogares': 'Households',
    'Tamaño medio del hogar': 'Median House Size',
    'Tasa de crecimiento demográfico 2021 (%)': 'Demographic Growth Rate',
    'Renta neta media anual de los hogares Urban Audit (2020)': 'Median Income',
    'Paro registrado (nº de personas registradas en SEPE en febrero 2023)': 'Median Unemployment Rate',
    'Año medio de construcción de inmuebles de uso residencial ': 'Median House Age',
    'No sabe leer ni escribir o sin estudios': 'Education Level',
    'Superficie media construida (m2) inmuebles de uso residencial  ': 'Average House Size',
    'Número de inmuebles de uso residencial  ': 'Number of Houses',
    'Valor catastral medio de los bienes inmuebles: personas físicas en miles de €': 'Cadastral Value of Properties',
    'En centros PRIVADOS CONCERTADOS': 'Number of Concertado Schools',
    'En centros PRIVADOS SIN CONCIERTO': 'Number of Private Schools',
    'En centros PÚBLICOS': 'Number of Public Schools'
}

socio_df.rename(columns=new_column_names, inplace=True)

print(socio_df.columns)


Index(['District', 'Surface', 'Density', 'Population Median Age',
       'People with Spanish Nationality', 'People with Foreign Nationality',
       'Households', 'Median House Size', 'Demographic Growth Rate',
       'Median Income', 'Median Unemployment Rate', 'Median House Age',
       'Education Level', 'Primaria incompleta', 'Average House Size',
       'Number of Houses', 'Cadastral Value of Properties',
       'Number of Concertado Schools', 'Number of Private Schools',
       'Number of Public Schools'],
      dtype='object', name=' ')


In [31]:

socio_df['District'] = socio_df['District'].astype(str)

In [32]:

if 'District' in socio_df.columns and issubclass(socio_df['District'].dtype.type, str):
    socio_df['District'] = socio_df['District'].str.capitalize()
else:
    print("\nError: 'District' column not found or is not of type string.")

print("\nDataFrame after converting 'District' to string and capitalization:")
socio_df.head()


Error: 'District' column not found or is not of type string.

DataFrame after converting 'District' to string and capitalization:


,District,Surface,Density,Population Median Age,People with Spanish Nationality,People with Foreign Nationality,Households,Median House Size,Demographic Growth Rate,Median Income,Median Unemployment Rate,Median House Age,Education Level,Primaria incompleta,Average House Size,Number of Houses,Cadastral Value of Properties,Number of Concertado Schools,Number of Private Schools,Number of Public Schools
0,Ciudad,60445.52,55.255228,44.37,0.82918,0.141,1.0,2.501912,-0.78,43003.0,148294.0,1973.206107,0.040039,0.101188,115.48855,1502190.0,90.421527,0.421171,0.181379,0.39745
1,Centro,522.82,267.179909,44.02,0.733755,0.266202,0.053231,1.99754,-1.11,36984.0,0.04456,1926.0,0.023226,0.063849,0.927942,0.05419,1.257932,0.360149,0.101329,0.538522
2,Arganzuela,646.22,237.231902,45.4,0.889494,0.106155,0.050032,2.322374,-1.05,45310.0,0.038592,1976.285714,0.020411,0.068992,0.833725,0.050177,0.926003,0.291119,0.194917,0.513964
3,Retiro,546.62,215.722074,47.58,0.909878,0.088027,0.037356,2.397897,-0.75,56302.0,0.023069,1964.666667,0.015502,0.051691,1.118437,0.03665,1.452984,0.551045,0.124804,0.324151
4,Salamanca,539.24,270.198798,46.23,0.842645,0.155674,0.048139,2.300152,-0.38,60401.0,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171


### Merging the datasets

Before merging the datasets, we want to ensure that everything is standardized

Socio_df has ciudad

In [33]:
socio_df = socio_df[socio_df['District'] != 'Ciudad']

print(socio_df['District'].unique())


['Centro' 'Arganzuela' 'Retiro' 'Salamanca' 'Chamartin' 'Tetuan'
 'Chamberi' 'Fuencarral-El Pardo' 'Moncloa-Aravaca' 'Latina' 'Carabanchel'
 'Usera' 'Puente de Vallecas' 'Moratalaz' 'Ciudad Lineal' 'Hortaleza'
 'Villaverde' 'Villa de Vallecas' 'Vicalvaro' 'San Blas-Canillejas'
 'Barajas']


In [34]:
print(data['District'].unique())

['Salamanca' 'Hortaleza' 'Chamartin' 'Tetuan' 'Moncloa-Aravaca' 'Centro'
 'Chamberi' 'Retiro' 'Fuencarral-El Pardo' 'Carabanchel' 'Arganzuela'
 'San Blas-Canillejas' 'Ciudad Lineal' 'Latina' 'Usera' 'Vicalvaro'
 'Villaverde' 'Villa de Vallecas' 'Puente de Vallecas' 'Barajas'
 'Moratalaz']


**Standardize district names**

In [35]:
socio_df['District'] = socio_df['District'].str.strip().str.title()
data['District'] = data['District'].str.strip().str.title()

print("Unique districts in original after cleanup:", data['District'].nunique())
print("Unique districts in new dataset after cleanup:", socio_df['District'].nunique())


Unique districts in original after cleanup: 21
Unique districts in new dataset after cleanup: 21


<ipython-input-35-e1904168c5a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socio_df['District'] = socio_df['District'].str.strip().str.title()


In [36]:
def standardize_district_names(df):
    df['District'] = df['District'].str.replace('-', ' ').str.strip().str.title()
    return df

data = standardize_district_names(data)
socio_df = standardize_district_names(socio_df)

print("Real Estate Districts:", sorted(data['District'].unique()))
print("Socio-Demographic Districts:", sorted(socio_df['District'].unique()))


Real Estate Districts: ['Arganzuela', 'Barajas', 'Carabanchel', 'Centro', 'Chamartin', 'Chamberi', 'Ciudad Lineal', 'Fuencarral El Pardo', 'Hortaleza', 'Latina', 'Moncloa Aravaca', 'Moratalaz', 'Puente De Vallecas', 'Retiro', 'Salamanca', 'San Blas Canillejas', 'Tetuan', 'Usera', 'Vicalvaro', 'Villa De Vallecas', 'Villaverde']
Socio-Demographic Districts: ['Arganzuela', 'Barajas', 'Carabanchel', 'Centro', 'Chamartin', 'Chamberi', 'Ciudad Lineal', 'Fuencarral El Pardo', 'Hortaleza', 'Latina', 'Moncloa Aravaca', 'Moratalaz', 'Puente De Vallecas', 'Retiro', 'Salamanca', 'San Blas Canillejas', 'Tetuan', 'Usera', 'Vicalvaro', 'Villa De Vallecas', 'Villaverde']


In [37]:
real_estate_districts = set(data['District'].unique())
socio_districts = set(socio_df['District'].unique())

common_districts = real_estate_districts & socio_districts
print("Common districts in both datasets:", common_districts)

only_in_real_estate = real_estate_districts - socio_districts
print("Districts only in real estate data:", only_in_real_estate)

only_in_socio = socio_districts - real_estate_districts
print("Districts only in socio-demographic data:", only_in_socio)

if real_estate_districts == socio_districts:
    print("Both datasets have exactly the same districts.")
else:
    print("There are differences in the districts listed in each dataset.")



Common districts in both datasets: {'Salamanca', 'Vicalvaro', 'Villa De Vallecas', 'Arganzuela', 'Usera', 'Fuencarral El Pardo', 'Hortaleza', 'Latina', 'Puente De Vallecas', 'Tetuan', 'Villaverde', 'Centro', 'Chamartin', 'Chamberi', 'San Blas Canillejas', 'Ciudad Lineal', 'Retiro', 'Moratalaz', 'Barajas', 'Carabanchel', 'Moncloa Aravaca'}
Districts only in real estate data: set()
Districts only in socio-demographic data: set()
Both datasets have exactly the same districts.


Check any discreprencies in district names:

Was encountering issues with the following districts: {'Ciudad Lineal', 'Villa De Vallecas', 'Puente De Vallecas', 'Fuencarral-El Pardo', 'San Blas-Canillejas', 'Moncloa-Aravaca'

In [38]:
enriched_data = pd.merge(data, socio_df, on='District', how='left')

In [39]:
enriched_data.head()

,url,listingUrl,title,id,price,baths,rooms,sqft,description,address,...,Median Unemployment Rate,Median House Age,Education Level,Primaria incompleta,Average House Size,Number of Houses,Cadastral Value of Properties,Number of Concertado Schools,Number of Private Schools,Number of Public Schools
0,https://www.idealista.com/inmueble/104027174/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Villanueva,104027174,1920000,3,3,183,residencia única con acabados de la más alta c...,"Recoletos, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
1,https://www.idealista.com/inmueble/102321942/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Núñez de Balboa,102321942,1995000,3,3,170,"preciosa reforma a estrenar, con terrazas y ga...","Castellana, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
2,https://www.idealista.com/inmueble/103334142/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Conde Orgaz-Piovera,103334142,1300000,3,4,270,de salas consultores inmobiliarios mav02679 le...,"Hortaleza, Madrid",...,0.04663,1986.833333,0.031129,0.083041,1.333466,0.056181,1.196373,0.323478,0.362814,0.313708
3,https://www.idealista.com/inmueble/104161987/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Nueva España,104161987,1650000,3,3,248,gilmar real estate vende espectacular piso en ...,"Chamartín, Madrid",...,0.025685,1969.666667,0.012234,0.039272,1.166061,0.045221,1.648696,0.457255,0.336839,0.205906
4,https://www.idealista.com/inmueble/103989666/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Claudio Coello,103989666,1590750,3,3,116,descubre esta impresionante vivienda exterior ...,"Recoletos, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171


Check whether they merged correctly

In [40]:

print("Unique districts in original DataFrame:", data['District'].nunique())
print("Unique districts in merged DataFrame:", enriched_data['District'].nunique())

print("Districts in original not in merged:", set(data['District']) - set(enriched_data['District']))
print("Districts in merged not in original:", set(enriched_data['District']) - set(data['District']))


Unique districts in original DataFrame: 21
Unique districts in merged DataFrame: 21
Districts in original not in merged: set()
Districts in merged not in original: set()


In [41]:
import pandas as pd
import numpy as np

numeric_columns = [
    'Surface', 'Density', 'Population Median Age',
    'People with Spanish Nationality', 'People with Foreign Nationality',
    'Households', 'Median House Size', 'Demographic Growth Rate',
    'Median Income', 'Median Unemployment Rate', 'Median House Age',
    'Education Level', 'Average House Size', 'Number of Houses',
    'Cadastral Value of Properties', 'Number of Concertado Schools',
    'Number of Private Schools', 'Number of Public Schools'
]

for column in numeric_columns:
    enriched_data[column] = pd.to_numeric(enriched_data[column], errors='coerce')

print(enriched_data.dtypes)

print(enriched_data.isna().sum())


url                                 object
listingUrl                          object
title                               object
id                                   int64
price                                int64
baths                                int64
rooms                                int64
sqft                                 int64
description                         object
address                             object
typology                            object
advertiserProfessionalName          object
advertiserName                      object
Terrace                               bool
Garage                                bool
Pool                                  bool
Garden                                bool
Exterior                              bool
RecentlyRenovated                     bool
HighFloor                             bool
NaturalLight                          bool
Views                                 bool
ClimateControl                        bool
EnergyEffic

In [42]:
enriched_data.head()

,url,listingUrl,title,id,price,baths,rooms,sqft,description,address,...,Median Unemployment Rate,Median House Age,Education Level,Primaria incompleta,Average House Size,Number of Houses,Cadastral Value of Properties,Number of Concertado Schools,Number of Private Schools,Number of Public Schools
0,https://www.idealista.com/inmueble/104027174/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Villanueva,104027174,1920000,3,3,183,residencia única con acabados de la más alta c...,"Recoletos, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
1,https://www.idealista.com/inmueble/102321942/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Núñez de Balboa,102321942,1995000,3,3,170,"preciosa reforma a estrenar, con terrazas y ga...","Castellana, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
2,https://www.idealista.com/inmueble/103334142/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Conde Orgaz-Piovera,103334142,1300000,3,4,270,de salas consultores inmobiliarios mav02679 le...,"Hortaleza, Madrid",...,0.046630,1986.833333,0.031129,0.083041,1.333466,0.056181,1.196373,0.323478,0.362814,0.313708
3,https://www.idealista.com/inmueble/104161987/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en Nueva España,104161987,1650000,3,3,248,gilmar real estate vende espectacular piso en ...,"Chamartín, Madrid",...,0.025685,1969.666667,0.012234,0.039272,1.166061,0.045221,1.648696,0.457255,0.336839,0.205906
4,https://www.idealista.com/inmueble/103989666/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Claudio Coello,103989666,1590750,3,3,116,descubre esta impresionante vivienda exterior ...,"Recoletos, Madrid",...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171


##**Step 5. Data Cleaning**

Remove the following columns:


*   ID
*   Url
*   Listing url
*   Title
*   AdvertiserProfessionalName
*   AdvertiserName
*   Debug_district
*   Is_Outlier

In [43]:
enriched_data.columns

Index(['url', 'listingUrl', 'title', 'id', 'price', 'baths', 'rooms', 'sqft',
       'description', 'address', 'typology', 'advertiserProfessionalName',
       'advertiserName', 'Terrace', 'Garage', 'Pool', 'Garden', 'Exterior',
       'RecentlyRenovated', 'HighFloor', 'NaturalLight', 'Views',
       'ClimateControl', 'EnergyEfficient', 'Security', 'Accessible',
       'SouthFacing', 'EastFacing', 'NorthFacing', 'WestFacing', 'NearSchool',
       'NearAmenities', 'NearMetro', 'NearLandmark', 'District',
       'debug_district', 'price_per_sqft', 'is_outlier', 'Surface', 'Density',
       'Population Median Age', 'People with Spanish Nationality',
       'People with Foreign Nationality', 'Households', 'Median House Size',
       'Demographic Growth Rate', 'Median Income', 'Median Unemployment Rate',
       'Median House Age', 'Education Level', 'Primaria incompleta',
       'Average House Size', 'Number of Houses',
       'Cadastral Value of Properties', 'Number of Concertado Schools',

In [44]:
columns_to_remove = ['id', 'url', 'listingUrl', 'title', 'advertiserProfessionalName', 'advertiserName', 'debug_district', 'is_outlier']
enriched_data = enriched_data.drop(columns=columns_to_remove, errors='ignore')


In [45]:
enriched_data.head()

,price,baths,rooms,sqft,description,address,typology,Terrace,Garage,Pool,...,Median Unemployment Rate,Median House Age,Education Level,Primaria incompleta,Average House Size,Number of Houses,Cadastral Value of Properties,Number of Concertado Schools,Number of Private Schools,Number of Public Schools
0,1920000,3,3,183,residencia única con acabados de la más alta c...,"Recoletos, Madrid",Pisos,False,False,False,...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
1,1995000,3,3,170,"preciosa reforma a estrenar, con terrazas y ga...","Castellana, Madrid",Pisos,True,True,False,...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171
2,1300000,3,4,270,de salas consultores inmobiliarios mav02679 le...,"Hortaleza, Madrid",Pisos,True,True,True,...,0.046630,1986.833333,0.031129,0.083041,1.333466,0.056181,1.196373,0.323478,0.362814,0.313708
3,1650000,3,3,248,gilmar real estate vende espectacular piso en ...,"Chamartín, Madrid",Pisos,True,True,False,...,0.025685,1969.666667,0.012234,0.039272,1.166061,0.045221,1.648696,0.457255,0.336839,0.205906
4,1590750,3,3,116,descubre esta impresionante vivienda exterior ...,"Recoletos, Madrid",Pisos,False,True,False,...,0.025881,1952.833333,0.012778,0.042515,1.153072,0.050842,1.749829,0.655892,0.188937,0.155171


## **Visual Inspection**
Use histograms and Q-Q plots to see the distribution - check for normality


Export dataset

In [46]:
enriched_data.to_csv("TFG_finaldata.csv")